<a href="https://colab.research.google.com/github/Abinaya-J/Self_Driving_Car/blob/main/P3_Behavioral_Cloning_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Load data**

In [1]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

--2021-06-22 10:50:09--  https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 13.224.2.204, 13.224.2.59, 13.224.2.22, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|13.224.2.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333137665 (318M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 317.70M  93.1MB/s    in 3.4s    

2021-06-22 10:50:12 (92.6 MB/s) - ‘data.zip’ saved [333137665/333137665]



In [2]:
!unzip -qq data.zip

### Create train and test data

In [ ]:
import cv2
import csv
import pandas as pd
import numpy as np

In [ ]:
alldata = pd.read_csv('data/driving_log.csv')
display(alldata[0:5])

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [ ]:
print(len(alldata))

8036


In [ ]:
#remove first row
#crop before and normalize
lines=[]
with open('data/driving_log.csv') as csvfile:
  reader = csv.reader(csvfile)
  for line in reader:
    lines.append(line)

In [ ]:
images=[]
steering_m=[]
for line in lines[1:]:
  for i in range(3):
    path=line[i]
    filename= path.split('/')[-1]
    newpath = 'data/IMG/' + filename
    image = cv2.imread(newpath)
    image = cv2.resize(image, dsize = (100, 260))
    images.append(image)
    sm = float(line[3])
    steering_m.append(sm)

In [ ]:
print(steering_m)

In [ ]:
X_train = np.array(images)
Y_train = np.array(steering_m)

In [ ]:
print(X_train[0].shape)

(160, 320, 3)


### Train and validate model

In [ ]:
import keras 
import pandas as pd
import tensorflow as tf
import cv2

#Processing input
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input

# Modelling
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D, Input
from keras.initializers import glorot_uniform
from keras.optimizers import SGD , Adam
from keras.callbacks import LearningRateScheduler
from keras import regularizers
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [ ]:
X_train = X_train/255

In [ ]:
def data():
    batch_size=32
    train_datagen = ImageDataGenerator(shear_range=0.2,
                                       rotation_range=20.,
                                       width_shift_range=0.3,
                                       height_shift_range=0.3,
                                       zoom_range=0.2)
    
    train_generator = train_datagen.flow(X_train,Y_train, batch_size=batch_size, shuffle=True)

    return train_generator

train_generator=data()

NameError: ignored

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', padding = 'same', input_shape=(160,320, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(128,activation= 'relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64,activation= 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(43,activation= 'softmax')])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = Adam(learning_rate=0.0006), metrics=["accuracy"])